In [2]:
from google.colab import drive
drive.mount('/content/drive/', force_remount = True)

Mounted at /content/drive/


In [29]:
# !pip install jiwer
import os
import glob
import pandas as pd
import os
# from jiwer import wer,cer
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', 0)

# TF-IDF Analysis

In [6]:
import re
string = "তিন গাছ তিন গাছ ছয় গাছ দেবে। "
re.findall(r"[\u0980-\u09FF]{2,}", string)

['তিন', 'গাছ', 'তিন', 'গাছ', 'ছয়', 'গাছ', 'দেবে']

In [5]:
test_df = pd.read_excel('/content/drive/MyDrive/local_speech/others/IUT_bhasha_bichitra/orig_iut_folds/fix_dataset/final/test/test.xlsx')
test_df

,file_name,External_ID,district,transcriptions,Split,annotator
0,test_barishal (1).wav,rec_21_audio_36.wav,barishal,তিন গাছ তিন গাছ ছয় গাছ দেবে। তুই আয় এই জাগা <>...,Test,nahar2709marzan@gmail.com
1,test_barishal (10).wav,rec_21_audio_45.wav,barishal,গরম লাগে <> কী আছে? কই দেহি। ও পড়বে হোনেন না? ...,Test,nahar2709marzan@gmail.com
2,test_barishal (100).wav,rec_24_audio_10.wav,barishal,চেড়তে চেড়তে আগা বাইরাইয়া যায়। গত বছছর যহন এতো ...,Test,nahar2709marzan@gmail.com
3,test_barishal (101).wav,rec_24_audio_11.wav,barishal,"ওরে আর, ওয়া তো যেকালে পারছো হেইকালে গাছে ওটছো ...",Test,nahar2709marzan@gmail.com
4,test_barishal (102).wav,rec_24_audio_12.wav,barishal,"মোর এট্টা ইসাব, কী? চা আর খামু না অনেক খাইছি। ...",Test,nahar2709marzan@gmail.com
...,...,...,...,...,...,...
1677,test_tangail (95).wav,rec_39_audio_97.wav,tangail,সো ছবিটা আসলে দেখলে মানে ভাল্লাগবো অনেক। তারপর...,Test,zubaer71.bd@gmail.com
1678,test_tangail (96).wav,rec_39_audio_98.wav,tangail,পরে ইদানীং আমার কেমনে জানি মনে অইলো যে এই ছবিড...,Test,zubaer71.bd@gmail.com
1679,test_tangail (97).wav,rec_39_audio_99.wav,tangail,বলিউডের অন্যতম এক প্রভাবশালী ডিরেক্টর সঞ্জয় লী...,Test,zubaer71.bd@gmail.com
1680,test_tangail (98).wav,rec_39_audio_100.wav,tangail,<> মুটামুটি এই চারজন বাদে আর কেউ নেই। আর এই ছব...,Test,zubaer71.bd@gmail.com


In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import re


bangla_stopwords = [
    'অতএব', 'অথচ', 'অথবা', 'অনুযায়ী', 'অনেক', 'অনেকে', 'অনেকেই', 'অন্তত', 'অন্য', 'অবধি',
    'অবশ্য', 'অর্থাত', 'আই', 'আগামী', 'আগে', 'আগেই', 'আছে', 'আজ', 'আদ্যভাগে', 'আপনার',
    'আপনি', 'আবার', 'আমরা', 'আমাকে', 'আমাদের', 'আমার', 'আমি', 'আর', 'আরও', 'ই', 'ইত্যাদি',
    'ইহা', 'উচিত', 'উত্তর', 'উনি', 'উপর', 'উপরে', 'এ', 'এঁদের', 'এঁরা', 'এই', 'একই', 'একটি',
    'একবার', 'একে', 'এক্', 'এখন', 'এখনও', 'এখানে', 'এখানেই', 'এটা', 'এটাই', 'এটি', 'এত',
    'এতটাই', 'এতে', 'এদের', 'এব', 'এবং', 'এবার', 'এমন', 'এমনকী', 'এমনি', 'এর', 'এরা', 'এল',
    'এস', 'এতে', 'ঐ', 'ও', 'ওঁদের', 'ওঁর', 'ওঁরা', 'ওই', 'ওকে', 'ওখানে', 'ওদের', 'ওর', 'ওরা',
    'কখনও', 'কত', 'কবে', 'কমনে', 'কয়েক', 'কয়েকটি', 'করছে', 'করছেন', 'করতে', 'করবে', 'করবেন',
    'করলে', 'করলেন', 'করা', 'করাই', 'করায়', 'করার', 'করি', 'করিতে', 'করিয়া', 'করিয়ে', 'করে',
    'করেই', 'করেছিলেন', 'করেছে', 'করেছেন', 'করেন', 'কাউকে', 'কাছ', 'কাছে', 'কাজ', 'কাজে',
    'কারও', 'কারণ', 'কি', 'কিংবা', 'কিছু', 'কিছুই', 'কিন্তু', 'কী', 'কে', 'কেউ', 'কেউই',
    'কেখা', 'কেন', 'কোটি', 'কোন', 'কোনও', 'কোনো', 'ক্ষেত্রে', 'কয়েক', 'খুব', 'গিয়ে', 'গিয়েছে',
    'গিয়ে', 'গুলি', 'গেছে', 'গেল', 'গেলে', 'গোটা', 'চলে', 'চান', 'চায়', 'চার', 'চালু', 'চেয়ে',
    'চেষ্টা', 'ছাড়া', 'ছাড়াও', 'ছিল', 'ছিলেন', 'জন', 'জনকে', 'জনের', 'জন্য', 'জন্যও', 'জে',
    'জানতে', 'জানা', 'জানানো', 'জানায়', 'জানিয়ে', 'জানিয়েছে', 'জে', 'টি', 'ঠিক', 'তখন', 'তত',
    'তথা', 'তবু', 'তবে', 'তা', 'তাঁকে', 'তাঁদের', 'তাঁর', 'তাঁরা', 'তাঁাহারা', 'তাই', 'তাও', 'তাকে',
    'তাতে', 'তাদের', 'তার', 'তারপর', 'তারা', 'তারৈ', 'তাহলে', 'তাহা', 'তাহাতে', 'তাহার', 'তিনঐ',
    'তিনি', 'তিনিও', 'তুমি', 'তুলে', 'তেমন', 'তো', 'তোমার', 'থাকবে', 'থাকবেন', 'থাকা', 'থাকায়',
    'থাকে', 'থাকেন', 'থেকে', 'থেকেই', 'থেকেও', 'দিকে', 'দিতে', 'দিন', 'দিয়ে', 'দিয়েছে', 'দিয়েছেন',
    'দিলেন', 'দু', 'দুই', 'দুটি', 'দুটো', 'দেওয়া', 'দেওয়ার', 'দেওয়া', 'দেখতে', 'দেখা', 'দেখে',
    'দেন', 'দেয়', 'দ্বারা', 'ধরা', 'ধরে', 'ধামার', 'নতুন', 'নয়', 'না', 'নাই', 'নাকি', 'নাগাদ',
    'নানা', 'নিজে', 'নিজেই', 'নিজেদের', 'নিজের', 'নিতে', 'নিয়ে', 'নিয়ে', 'নেই', 'নেওয়া', 'নেওয়ার',
    'নেওয়া', 'নয়', 'পক্ষে', 'পর', 'পরে', 'পরেই', 'পরেও', 'পর্যন্ত', 'পাওয়া', 'পাচ', 'পারি',
    'পারে', 'পারেন', 'পি', 'পেয়ে', 'পেয়্র্', 'প্রতি', 'প্রথম', 'প্রভৃতি', 'প্রযন্ত', 'প্রায়',
    'প্রায়', 'ফলে', 'ফিরে', 'ফের', 'বক্তব্য', 'বদলে', 'বন', 'বরং', 'বলতে', 'বলল', 'বললেন', 'বলা',
    'বলে', 'বলেছেন', 'বলেন', 'বসে', 'বহু', 'বা', 'বাদে', 'বার', 'বি', 'বিনা', 'বিভিন্ন', 'বিশেষ',
    'বিষয়টি', 'বেশ', 'বেশি', 'ব্যবহার', 'ব্যাপারে', 'ভাবে', 'ভাবেই', 'মতো', 'মতোই', 'মধ্যে',
    'মধ্যে', 'মধ্যেই', 'মধ্যেও', 'মনে', 'মাত্র', 'মাধ্যমে', 'মোট', 'মোটেই', 'যখন', 'যত',
    'যতটা', 'যথেষ্ট', 'যদি', 'যদিও', 'যা', 'যাঁর', 'যাঁরা', 'যাওয়া', 'যাওয়ার', 'যাওয়া',
    'যাকে', 'যাচ্ছে', 'যাতে', 'যাদের', 'যান', 'যাবে', 'যায়', 'যার', 'যারা', 'যিনি', 'যে',
    'যেখানে', 'যেতে', 'যেন', 'যেমন', 'র', 'রকম', 'রয়েছে', 'রাখা', 'রেখে', 'লক্ষ', 'শুধু',
    'শুরু', 'সঙ্গে', 'সঙ্গেও', 'সব', 'সবার', 'সমস্ত', 'সম্প্রতি', 'সহ', 'সহিত', 'সাধারণ', 'সামনে',
    'সি', 'সুতরাং', 'সে', 'সেই', 'সেখান', 'সেখানে', 'সেটা', 'সেটাই', 'সেটাও', 'সেটি', 'স্পষ্ট',
    'স্বয়ং', 'হইতে', 'হইবে', 'হইয়া', 'হওয়া', 'হওয়ায়', 'হওয়ার', 'হচ্ছে', 'হত', 'হতে',
    'হতেই', 'হন', 'হবে', 'হবেন', 'হয়', 'হয়তো', 'হয়নি', 'হয়ে', 'হয়েই', 'হয়েছিল', 'হয়েছে',
    'হয়েছেন', 'হল', 'হলে', 'হলেই', 'হলেও', 'হলো', 'হাজার', 'হিসাবে', 'হৈলে', 'হোক', 'হয়'
]


def bangla_tokenizer(text):
    return re.findall(r"[\u0980-\u09FF\u200d]{2,}", text)

district_tfidf_results = {}

for district, group in test_df.groupby('district'):
    transcriptions = group['transcriptions'].astype(str)

    vectorizer = TfidfVectorizer(
        max_features=1000,
        stop_words=bangla_stopwords,
        tokenizer=bangla_tokenizer,
        token_pattern=None
    )

    tfidf_matrix = vectorizer.fit_transform(transcriptions)

    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out(), index=group['file_name'])

    district_tfidf_results[district] = tfidf_df

district_tfidf_results['rangpur']


,অংপুরের,অই,অইগলা,অইতন,অইযে,অইল্লা,অটে,অন্যরকম,অপর,অবস্থা,...,হেইল,হেনে,হেয়া,হো,হোটেল,হোটেলত,হ্যা,হ্যাঁ,হয়তোবা,হয়া
file_name,,,,,,,,,,,,,,,,,,,,,
test_rangpur (1).wav,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
test_rangpur (10).wav,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
test_rangpur (100).wav,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
test_rangpur (101).wav,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
test_rangpur (102).wav,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
test_rangpur (95).wav,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
test_rangpur (96).wav,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
test_rangpur (97).wav,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
district_tfidf_results['barishal']

,অইছে,অইতাম,অইতে,অইতো,অইবে,অইবেই,অইল,অইলে,অইলো,অইয়া,...,হোনে,হোনেন,হোম,হোমিপ্যাতিতে,হ্যা,হ্যাগো,হ্যার,হ্যারপর,হ্যারা,হ্যাষে
file_name,,,,,,,,,,,,,,,,,,,,,
test_barishal (1).wav,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.126978,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
test_barishal (10).wav,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.178432,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
test_barishal (100).wav,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
test_barishal (101).wav,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
test_barishal (102).wav,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
test_barishal (95).wav,0.114365,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
test_barishal (96).wav,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.104701,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
test_barishal (97).wav,0.000000,0.0,0.0,0.0,0.282004,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
term_importance = district_tfidf_results['barishal'].sum().sort_values(ascending=False)
term_importance.head(20)

,0
এহন,3.497085
মোর,3.467270
মুই,3.221950
কাম,3.068827
কয়,2.950863
এট্টু,2.939520
তয়,2.777511
দেহি,2.622195
হেই,2.518894
লাগে,2.510776


In [ ]:
def get_high_low_tfidf(district_tfidf_results, top_n=10):
    high_tfidf = {}
    low_tfidf = {}

    for district, tfidf_df in district_tfidf_results.items():
        sorted_tfidf = tfidf_df.sum(axis=0).sort_values(ascending=False)  # Sum TF-IDF per word
        high_tfidf[district] = sorted_tfidf.head(top_n).index.tolist()
        low_tfidf[district] = sorted_tfidf.tail(top_n).index.tolist()

    return high_tfidf, low_tfidf


high_tfidf_words, low_tfidf_words = get_high_low_tfidf(district_tfidf_results)

for district, tfidf_df in district_tfidf_results.items():
    print(f"\n District Name: {district}")

    high_words_scores = tfidf_df.sum(axis=0).sort_values(ascending=False).head(10)
    print("\n High TF-IDF Words: ")
    for word, score in high_words_scores.items():
        print(f"  {word}: {score:.4f}")

    low_words_scores = tfidf_df.sum(axis=0).sort_values(ascending=False).tail(10)
    print("\n Low TF-IDF Words: ")
    for word, score in low_words_scores.items():
        print(f"  {word}: {score:.4f}")
    print("-" * 50)



 District Name: barishal

 High TF-IDF Words: 
  এহন: 3.4971
  মোর: 3.4673
  মুই: 3.2220
  কাম: 3.0688
  কয়: 2.9509
  এট্টু: 2.9395
  তয়: 2.7775
  দেহি: 2.6222
  হেই: 2.5189
  লাগে: 2.5108

 Low TF-IDF Words: 
  হেডু: 0.1281
  প্রয়জোন: 0.1272
  ব্যাডারাই: 0.1272
  শরম: 0.1272
  মরতে: 0.1272
  মরবে: 0.1272
  ভাবি: 0.1272
  বুজ্জি: 0.1221
  হেইতো: 0.1221
  সোংসার: 0.1221
--------------------------------------------------

 District Name: chittagong

 High TF-IDF Words: 
  আঁই: 7.1621
  দে: 7.1399
  আঁর: 6.3386
  তুই: 6.1999
  ওডি: 5.0957
  দি: 4.9902
  গরি: 4.7565
  এহন: 4.4732
  হতা: 4.3543
  লই: 4.1207

 Low TF-IDF Words: 
  বাঁশবাড়িয়া: 0.1514
  আশতোষ: 0.1514
  আসছে: 0.1506
  বারের: 0.1506
  ভাইও: 0.1444
  আবি: 0.1276
  ইয়ততুন: 0.1067
  বানাইয়িদে: 0.1067
  উইটটন: 0.1067
  আসিয়রে: 0.1067
--------------------------------------------------

 District Name: habiganj

 High TF-IDF Words: 
  বালা: 4.0877
  কিতা: 3.8273
  লাগে: 3.8079
  একটা: 2.8887
  এরফরে: 2.8711
  ইতা: 2.7099
  বাই: 2.669